In [49]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch

In [50]:
# this takes the data from the weather and mathes it up with the data from the valley fever rates

weather = pd.read_csv('../../data/weather_datasets/one_hot_weather_06_02_24.csv')
rates = pd.read_csv('../../data/valley_fever_rates/valley_fever_cases_05_31_24.csv')

In [51]:
print(f'Weather shape: {weather.shape}')
print(f'Rates shape: {rates.shape}')

Weather shape: (19241397, 75)
Rates shape: (1056, 4)


In [52]:

weather = weather.drop(['Wthrmain','Wthrdesc'], axis=1)

In [53]:
weather.head()

,County,Lat,Lon,Temp,Dewpt,Feels,Tmin,Tmax,Pres,Humid,...,Snow.1,Sqls,Thndr.1,Thdrzl,Thhr,Thldrz,Thlr,Thr,Torn.1,Vhr
0,Alameda,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alameda,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alameda,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alameda,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,68,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alameda,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
rates = rates.drop('Unnamed: 0',axis=1)
rates.head()


,County,Year,Cases
0,ALAMEDA,2001,7
1,ALAMEDA,2002,2
2,ALAMEDA,2003,9
3,ALAMEDA,2004,12
4,ALAMEDA,2005,11


In [55]:
# i made the weather start in 2000 because the output for 2001 of the rates will contain data from september 2000 to august 2001
weather = weather.loc[weather['Year'] >= 2000]
weather = weather.loc[weather['Year'] <= 2022]
weather.shape

(9750922, 73)

In [56]:
print(weather['County'].unique())
print(rates['County'].unique())

['Alameda' 'Amador' 'Berkeley' 'Butte' 'Calaveras' 'Colusa' 'ContraCosta'
 'ElDorado' 'Fresno' 'Humboldt' 'Imperial' 'Kern' 'Lassen' 'LongBeach'
 'Madera' 'Marin' 'Mendocino' 'Merced' 'Monterey' 'Napa' 'Nevada' 'Orange'
 'Pasadena' 'Placer' 'Riverside' 'Sacramento' 'SanBenito' 'SanBernardino'
 'SanDiego' 'SanFrancisco' 'SanJoaquin' 'SanLuisObispo' 'SanMateo'
 'SantaBarbara' 'SantaClara' 'SantaCruz' 'Siskiyou' 'Solano' 'Sonoma'
 'Stanislaus' 'Sutter' 'Tehama' 'Tulare' 'Tuolumne' 'Ventura' 'Yolo'
 'Yuba' 'Kings']
['ALAMEDA' 'AMADOR' 'BERKELEY' 'BUTTE' 'CALAVERAS' 'COLUSA' 'CONTRA COSTA'
 'EL DORADO' 'FRESNO' 'HUMBOLDT' 'IMPERIAL' 'KERN' 'KINGS' 'LASSEN'
 'LONG BEACH' 'MADERA' 'MARIN' 'MENDOCINO' 'MERCED' 'MONTEREY' 'NAPA'
 'NEVADA' 'ORANGE' 'PASADENA' 'PLACER' 'RIVERSIDE' 'SACRAMENTO'
 'SAN BENITO' 'SAN BERNARDINO' 'SAN DIEGO' 'SAN FRANCISCO' 'SAN JOAQUIN'
 'SAN LUIS OBISPO' 'SAN MATEO' 'SANTA BARBARA' 'SANTA CLARA' 'SANTA CRUZ'
 'SISKIYOU' 'SOLANO' 'SONOMA' 'STANISLAUS' 'SUTTER' 'TEHAMA

In [57]:
#relabel all of the counties so it is in a better format

rates['County'] = rates['County'].replace({
    'ALAMEDA': 'Alameda',
    'AMADOR': 'Amador',
    'BERKELEY': 'Berkeley',
    'BUTTE': 'Butte',
    'CALAVERAS': 'Calaveras',
    'COLUSA': 'Colusa',
    'CONTRA COSTA': 'ContraCosta',
    'EL DORADO': 'ElDorado',
    'FRESNO': 'Fresno',
    'HUMBOLDT': 'Humboldt',
    'IMPERIAL': 'Imperial',
    'KERN': 'Kern',
    'LASSEN': 'Lassen',
    'LONG BEACH': 'LongBeach',
    'MADERA': 'Madera',
    'MARIN': 'Marin',
    'MENDOCINO': 'Mendocino',
    'MERCED': 'Merced',
    'MONTEREY': 'Monterey',
    'NAPA': 'Napa',
    'NEVADA': 'Nevada',
    'ORANGE': 'Orange',
    'PASADENA': 'Pasadena',
    'PLACER': 'Placer',
    'RIVERSIDE': 'Riverside',
    'SACRAMENTO': 'Sacramento',
    'SAN BENITO': 'SanBenito',
    'SAN BERNARDINO': 'SanBernardino',
    'SAN DIEGO': 'SanDiego',
    'SAN FRANCISCO': 'SanFrancisco',
    'SAN JOAQUIN': 'SanJoaquin',
    'SAN LUIS OBISPO': 'SanLuisObispo',
    'SAN MATEO': 'SanMateo',
    'SANTA BARBARA': 'SantaBarbara',
    'SANTA CLARA': 'SantaClara',
    'SANTA CRUZ': 'SantaCruz',
    'SISKIYOU': 'Siskiyou',
    'SOLANO': 'Solano',
    'SONOMA': 'Sonoma',
    'STANISLAUS': 'Stanislaus',
    'SUTTER': 'Sutter',
    'TEHAMA': 'Tehama',
    'TULARE': 'Tulare',
    'TUOLUMNE': 'Tuolumne',
    'VENTURA': 'Ventura',
    'YOLO': 'Yolo',
    'YUBA': 'Yuba',
    'KINGS': 'Kings'
})
print(rates['County'].unique())

['Alameda' 'Amador' 'Berkeley' 'Butte' 'Calaveras' 'Colusa' 'ContraCosta'
 'ElDorado' 'Fresno' 'Humboldt' 'Imperial' 'Kern' 'Kings' 'Lassen'
 'LongBeach' 'Madera' 'Marin' 'Mendocino' 'Merced' 'Monterey' 'Napa'
 'Nevada' 'Orange' 'Pasadena' 'Placer' 'Riverside' 'Sacramento'
 'SanBenito' 'SanBernardino' 'SanDiego' 'SanFrancisco' 'SanJoaquin'
 'SanLuisObispo' 'SanMateo' 'SantaBarbara' 'SantaClara' 'SantaCruz'
 'Siskiyou' 'Solano' 'Sonoma' 'Stanislaus' 'Sutter' 'Tehama' 'Tulare'
 'Tuolumne' 'Ventura' 'Yolo' 'Yuba']


In [58]:
# rates.to_csv('../../data/valley_fever_rates/valley_fever_cases_06_02_24.csv', index=False)

In [59]:
weather.head()

,County,Lat,Lon,Temp,Dewpt,Feels,Tmin,Tmax,Pres,Humid,...,Snow.1,Sqls,Thndr.1,Thdrzl,Thhr,Thldrz,Thlr,Thr,Torn.1,Vhr
184260,Alameda,37.601689,-121.719546,12.42,5.35,11.34,11.87,13.31,1016,62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184261,Alameda,37.601689,-121.719546,10.87,5.83,9.87,9.30,12.15,1017,71,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184262,Alameda,37.601689,-121.719546,10.18,5.95,9.21,8.30,11.66,1017,75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184263,Alameda,37.601689,-121.719546,7.26,3.13,4.58,5.40,8.98,1018,75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184264,Alameda,37.601689,-121.719546,6.84,2.72,4.65,5.20,9.01,1018,75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
rates.head()

,County,Year,Cases
0,Alameda,2001,7
1,Alameda,2002,2
2,Alameda,2003,9
3,Alameda,2004,12
4,Alameda,2005,11


In [61]:
weather.shape

(9750922, 73)

In [62]:
rates.shape

(1056, 3)

In [63]:
print(weather['Year'].unique())

[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021 2022]


In [64]:
for county, group in weather.groupby('County'):
    print(f"County: {county}, Number of unique years: {len(group['Year'].unique())}")


County: Alameda, Number of unique years: 23
County: Amador, Number of unique years: 23
County: Berkeley, Number of unique years: 23
County: Butte, Number of unique years: 23
County: Calaveras, Number of unique years: 23
County: Colusa, Number of unique years: 23
County: ContraCosta, Number of unique years: 23
County: ElDorado, Number of unique years: 23
County: Fresno, Number of unique years: 23
County: Humboldt, Number of unique years: 23
County: Imperial, Number of unique years: 23
County: Kern, Number of unique years: 23
County: Kings, Number of unique years: 23
County: Lassen, Number of unique years: 23
County: LongBeach, Number of unique years: 23
County: Madera, Number of unique years: 23
County: Marin, Number of unique years: 23
County: Mendocino, Number of unique years: 23
County: Merced, Number of unique years: 23
County: Monterey, Number of unique years: 23
County: Napa, Number of unique years: 23
County: Nevada, Number of unique years: 23
County: Orange, Number of unique yea

In [65]:
print(len(weather['County'].unique()))

48


# math 
The weather dataset has a totatl of:
    - 9750922 instances
    - 48 counties
    - 23 years 
    - 365 days in a year
    - 24 hours in a day

so to verify that the number of instances is correct 
we can do 48 * 23 * 365 * 24 = 9671040


In [66]:
print(len(rates['County'].unique()))

48


In [67]:
for county, group in rates.groupby('County'):
    print(f"County: {county}, Number of unique years: {len(group['Year'].unique())}")

County: Alameda, Number of unique years: 22
County: Amador, Number of unique years: 22
County: Berkeley, Number of unique years: 22
County: Butte, Number of unique years: 22
County: Calaveras, Number of unique years: 22
County: Colusa, Number of unique years: 22
County: ContraCosta, Number of unique years: 22
County: ElDorado, Number of unique years: 22
County: Fresno, Number of unique years: 22
County: Humboldt, Number of unique years: 22
County: Imperial, Number of unique years: 22
County: Kern, Number of unique years: 22
County: Kings, Number of unique years: 22
County: Lassen, Number of unique years: 22
County: LongBeach, Number of unique years: 22
County: Madera, Number of unique years: 22
County: Marin, Number of unique years: 22
County: Mendocino, Number of unique years: 22
County: Merced, Number of unique years: 22
County: Monterey, Number of unique years: 22
County: Napa, Number of unique years: 22
County: Nevada, Number of unique years: 22
County: Orange, Number of unique yea

In [69]:
df = weather.copy()

df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df = df.drop(['Year', 'Month', 'Day'], axis=1)

x = torch.empty((0, 0))  
y = torch.empty(0)  

for year in range(2000, 2022):
    start_date = pd.Timestamp(year=year, month=9, day=1)
    end_date = pd.Timestamp(year=year + 1, month=8, day=31)

    tmp = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    for county, group in tmp.groupby('County'):
        
        rate = rates[(rates['County'] == county) & (rates['Year'] == end_date.year)]['Cases']
        print(rate)
        
        if not rate.empty:
            group = group.drop('County', axis=1)
            group = group.apply(pd.to_numeric, errors='coerce')
            x = torch.cat((x, torch.tensor(group.values, dtype=torch.float32)), dim=0)
            y = torch.cat((y, torch.tensor(rate.values, dtype=torch.float32)), dim=0)